## This example demonstrates XCSF hyperparameter tuning

In [1]:
import numpy as np
import optuna
from optuna.samplers import TPESampler
from optuna.visualization import plot_param_importances
from sklearn.compose import TransformedTargetRegressor
from sklearn.datasets import fetch_openml
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import xcsf

RANDOM_STATE: int = 1

np.random.seed(RANDOM_STATE)

### Load training and test data
Loads the kin8nm dataset from: https://www.openml.org/d/189

In [2]:
data = fetch_openml(data_id=189, as_frame=True, parser="auto")

data.frame.head()

,theta1,theta2,theta3,theta4,theta5,theta6,theta7,theta8,y
0,-0.015119,0.360741,0.469398,1.309675,0.988024,-0.025493,0.664071,0.062763,0.536524
1,0.360478,-0.301395,0.629183,-1.440146,-0.741637,-1.196749,-1.038444,-0.717461,0.308014
2,1.563238,-1.294753,0.078987,1.432937,1.149136,-1.292140,1.562988,-0.937731,0.518900
3,0.199485,0.901157,-1.356304,-0.080525,-0.976628,0.829894,-0.855649,0.930630,0.494151
4,0.659737,0.120552,-0.008756,0.648839,0.626832,-0.646539,1.318074,-0.899172,0.470218


In [3]:
data.frame.describe()

,theta1,theta2,theta3,theta4,theta5,theta6,theta7,theta8,y
count,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000,8192.000000
mean,0.009351,-0.007233,0.022792,-0.012945,0.007886,-0.009698,0.008402,0.001227,0.714283
std,0.904867,0.901496,0.900765,0.899186,0.906874,0.913243,0.909724,0.906098,0.263607
min,-1.570611,-1.570681,-1.570669,-1.570295,-1.570664,-1.570254,-1.570643,-1.568904,0.040165
25%,-0.770328,-0.778319,-0.749718,-0.780602,-0.770492,-0.802084,-0.776870,-0.777432,0.523584
50%,-0.002010,-0.015152,0.041346,-0.013617,0.009702,-0.003575,0.001338,0.002007,0.708822
75%,0.791179,0.779145,0.792391,0.760165,0.791203,0.781849,0.798014,0.780927,0.901354
max,1.570753,1.570657,1.570736,1.570169,1.570599,1.570597,1.570522,1.570237,1.458521


In [4]:
# numpy
X_orig = np.asarray(data.data, dtype=np.float64)
y_orig = np.asarray(data.target, dtype=np.float64)

# normalise inputs (zero mean and unit variance)
feature_scaler = StandardScaler()
X = feature_scaler.fit_transform(X_orig)

# reshape outputs into 2D arrays
if y_orig.ndim == 1:
    y_orig = y_orig.reshape(-1, 1)

# normalise outputs (zero mean and unit variance)
output_scaler = StandardScaler()
y = output_scaler.fit_transform(y_orig)

# split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=RANDOM_STATE
)

# 10% of training for validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.1, random_state=RANDOM_STATE
)

print(f"X_train shape = {np.shape(X_train)}")
print(f"y_train shape = {np.shape(y_train)}")
print(f"X_val shape = {np.shape(X_val)}")
print(f"y_val shape = {np.shape(y_val)}")
print(f"X_test shape = {np.shape(X_test)}")
print(f"y_test shape = {np.shape(y_test)}")

X_DIM: int = np.shape(X_train)[1]
Y_DIM: int = np.shape(y_train)[1]

X_train shape = (6634, 8)
y_train shape = (6634, 1)
X_val shape = (738, 8)
y_val shape = (738, 1)
X_test shape = (820, 8)
y_test shape = (820, 1)


### Initialise XCSF

In [5]:
# Due to the way XCSF sets parameters for conditions, actions, and predictions,
# it is necessary to set the desired parameters in a wrapper function.


def get_prediction(n_init: int) -> dict:
    """Returns parameters for initialising neural network predictions."""
    return {
        "type": "neural",
        "args": {
            "layer_0": {
                "type": "connected",
                "activation": "selu",
                "n_init": n_init,
                "evolve_weights": True,
                "evolve_functions": False,
                "evolve_connect": True,
                "evolve_neurons": False,
                "sgd_weights": True,
                "eta": 0.1,
                "evolve_eta": True,
                "eta_min": 1e-06,
                "momentum": 0.9,
                "decay": 0,
            },
            "layer_1": {
                "type": "connected",
                "activation": "selu",
                "n_init": Y_DIM,
                "evolve_weights": True,
                "evolve_functions": False,
                "evolve_connect": True,
                "evolve_neurons": False,
                "sgd_weights": True,
                "eta": 0.1,
                "evolve_eta": True,
                "eta_min": 1e-06,
                "momentum": 0.9,
                "decay": 0,
            },
        },
    }


def get_model() -> xcsf.XCS:
    """Returns a new XCS with baseline parameters."""
    return xcsf.XCS(
        x_dim=X_DIM,
        y_dim=Y_DIM,
        n_actions=1,
        omp_num_threads=12,
        random_state=RANDOM_STATE,
        pop_init=True,
        max_trials=200000,
        perf_trials=5000,
        pop_size=500,
        loss_func="mse",
        set_subsumption=False,
        theta_sub=100,
        e0=0.1,
        alpha=1,
        nu=20,
        beta=0.1,
        delta=0.1,
        theta_del=50,
        init_fitness=0.01,
        init_error=0,
        m_probation=10000,
        stateful=True,
        compaction=False,
        ea={
            "select_type": "roulette",
            "theta_ea": 50,
            "lambda": 2,
            "p_crossover": 0.8,
            "err_reduc": 1,
            "fit_reduc": 0.1,
            "subsumption": False,
            "pred_reset": False,
        },
        condition={
            "type": "tree_gp",
            "args": {
                "min_constant": 0,
                "max_constant": 1,
                "n_constants": 100,
                "init_depth": 5,
                "max_len": 10000,
            },
        },
        prediction=get_prediction(40),
    )

In [6]:
def get_callback():
    """Returns a new callback for early stoppping."""
    return xcsf.EarlyStoppingCallback(
        # note: PERF_TRIALS is considered an "epoch" for callbacks
        monitor="val",  # which metric to monitor: {"train", "val"}
        patience=20000,  # trials with no improvement after which training will be stopped
        restore_best=True,  # whether to make checkpoints and restore best population
        min_delta=0,  # minimum change to qualify as an improvement
        start_from=0,  # trials to wait before starting to monitor improvement
        verbose=True,  # whether to display when checkpoints are made
    )

### Sklearn pipeline

Note: sklearn pipelines do not allow the use of callback functions.

In [7]:
# split (original data) into training and test sets
X_tr, X_te, y_tr, y_te = train_test_split(
    X_orig, y_orig, test_size=0.1, random_state=RANDOM_STATE
)

model = get_model()
model.set_params(max_trials=50000)  # speed this demo up

pipe = make_pipeline(
    StandardScaler(),
    TransformedTargetRegressor(regressor=model, transformer=StandardScaler()),
)

pipe.fit(X_tr, y_tr)

2023-07-20T10:55:52.985 trials=5000 train=2.59709 pset=500.0 mset=243.8 mfrac=1.00
2023-07-20T10:55:53.520 trials=10000 train=0.31712 pset=500.0 mset=189.0 mfrac=0.83
2023-07-20T10:55:54.119 trials=15000 train=0.18845 pset=500.0 mset=352.0 mfrac=0.58
2023-07-20T10:55:54.937 trials=20000 train=0.20027 pset=500.0 mset=380.0 mfrac=0.63
2023-07-20T10:55:55.783 trials=25000 train=0.14154 pset=500.0 mset=384.2 mfrac=0.86
2023-07-20T10:55:56.669 trials=30000 train=0.13113 pset=500.0 mset=395.7 mfrac=0.48
2023-07-20T10:55:57.569 trials=35000 train=0.12875 pset=500.0 mset=407.1 mfrac=0.22
2023-07-20T10:55:58.494 trials=40000 train=0.12401 pset=500.0 mset=424.9 mfrac=1.00
2023-07-20T10:55:59.449 trials=45000 train=0.11988 pset=500.0 mset=427.0 mfrac=0.94
2023-07-20T10:56:00.425 trials=50000 train=0.11159 pset=500.0 mset=430.9 mfrac=1.00


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('transformedtargetregressor',
                 TransformedTargetRegressor(regressor=<xcsf.xcsf.XCS object at 0x7f797b4914b0>,
                                            transformer=StandardScaler()))])

In [8]:
train_score = pipe.score(X_tr, y_tr)
test_score = pipe.score(X_te, y_te)

print(f"train = {train_score}")
print(f"test = {test_score}")

train = 0.8593546109484066
test = 0.8514895158647242


### Sklearn grid search

In [9]:
# General parameters can be searched in the usual way

parameters = dict(beta=[0.1, 0.5])

# EA parameters require specifying a dict,
# but individual values can still be set because the other values are not changed.

parameters = dict(ea=[{"lambda": 2}, {"lambda": 10}])

# However, for actions, conditions, and predictions, the WHOLE dict must be specified
# for each value to try in the search. See Optuna below.

model = get_model()
model.set_params(max_trials=25000)  # speed this demo up

grid_search = GridSearchCV(
    estimator=model,
    param_grid=parameters,
    scoring="neg_mean_squared_error",
    cv=3,
)

grid_search.fit(X_train, y_train)

results = grid_search.cv_results_

for mean_score, std_score, params in zip(
    results["mean_test_score"], results["std_test_score"], results["params"]
):
    print("Mean Score:", -mean_score)
    print("Standard Deviation:", std_score)
    print("Parameters:", params)
    print("------------------------")

print("Best parameters: ", grid_search.best_params_)
print("Best score: ", -grid_search.best_score_)

2023-07-20T10:56:01.901 trials=5000 train=2.85196 pset=500.0 mset=246.2 mfrac=0.28
2023-07-20T10:56:02.433 trials=10000 train=0.31370 pset=500.0 mset=208.8 mfrac=0.54
2023-07-20T10:56:03.220 trials=15000 train=0.18522 pset=500.0 mset=403.6 mfrac=0.76
2023-07-20T10:56:04.416 trials=20000 train=0.15866 pset=500.0 mset=415.3 mfrac=0.99
2023-07-20T10:56:05.632 trials=25000 train=0.14775 pset=500.0 mset=407.4 mfrac=0.61
2023-07-20T10:56:06.687 trials=5000 train=2.52929 pset=500.0 mset=225.5 mfrac=0.34
2023-07-20T10:56:08.108 trials=10000 train=0.47156 pset=500.0 mset=180.3 mfrac=0.57
2023-07-20T10:56:09.028 trials=15000 train=0.25919 pset=500.0 mset=329.7 mfrac=0.48
2023-07-20T10:56:10.730 trials=20000 train=0.21262 pset=500.0 mset=345.8 mfrac=1.00
2023-07-20T10:56:11.978 trials=25000 train=0.16461 pset=500.0 mset=348.9 mfrac=0.25
2023-07-20T10:56:13.722 trials=5000 train=2.53970 pset=500.0 mset=251.3 mfrac=1.00
2023-07-20T10:56:14.497 trials=10000 train=0.31472 pset=500.0 mset=215.8 mfrac=

### Optuna

In [10]:
def objective(trial):
    """Measure a new hyperparameter combination."""
    # get new parameters to try
    e0: float = trial.suggest_float("e0", 0.08, 0.12, step=0.01)
    beta: float = trial.suggest_float("beta", 0.01, 0.19, step=0.02)
    n_init: int = trial.suggest_int("n_init", 1, 101, step=10)
    # get a new XCSF and set params - predictions must include the whole dict
    prediction: dict = get_prediction(n_init=n_init)
    model = get_model()
    model.set_params(e0=e0, beta=beta, prediction=prediction)
    # get a new early stopping callback
    callback = get_callback()
    # fit XCSF
    model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        callbacks=[callback],
        verbose=True,
    )
    # optimise validation MSE
    return model.score(X_val, y_val)


study = optuna.create_study(direction="minimize", sampler=TPESampler(seed=RANDOM_STATE))

[I 2023-07-20 10:56:46,725] A new study created in memory with name: no-name-cd14d920-ab09-402c-808d-fc52f1fcb706


In [11]:
study.optimize(objective, n_trials=10)

2023-07-20T10:56:47.340 trials=5000 train=0.69635 val=0.64039 pset=500.0 mset=258.8 mfrac=0.43
2023-07-20T10:56:47.341 EarlyStoppingCallback: 0.64039 best error at 5000 trials
2023-07-20T10:56:48.871 trials=10000 train=0.62533 val=0.60807 pset=500.0 mset=291.0 mfrac=1.00
2023-07-20T10:56:48.872 EarlyStoppingCallback: 0.60807 best error at 10000 trials
2023-07-20T10:56:50.802 trials=15000 train=0.60772 val=0.59613 pset=500.0 mset=387.6 mfrac=0.22
2023-07-20T10:56:50.803 EarlyStoppingCallback: 0.59613 best error at 15000 trials
2023-07-20T10:56:51.662 trials=20000 train=0.58156 val=0.64602 pset=500.0 mset=384.6 mfrac=0.01
2023-07-20T10:56:52.565 trials=25000 train=0.59017 val=0.59779 pset=500.0 mset=371.8 mfrac=0.35
2023-07-20T10:56:53.422 trials=30000 train=0.55726 val=0.59122 pset=500.0 mset=358.3 mfrac=0.77
2023-07-20T10:56:53.423 EarlyStoppingCallback: 0.59122 best error at 30000 trials
2023-07-20T10:56:54.180 trials=35000 train=0.59848 val=0.59155 pset=500.0 mset=351.6 mfrac=0.00
20

[I 2023-07-20 10:56:58,942] Trial 0 finished with value: 0.5770711109016021 and parameters: {'e0': 0.1, 'beta': 0.15000000000000002, 'n_init': 1}. Best is trial 0 with value: 0.5770711109016021.


2023-07-20T10:56:58.899 trials=70000 train=0.57828 val=0.62527 pset=500.0 mset=267.4 mfrac=0.00
2023-07-20T10:56:58.900 EarlyStoppingCallback: stopping
2023-07-20T10:56:58.900 EarlyStoppingCallback: restoring system from trial 45000 with error=0.57707
2023-07-20T10:56:59.921 trials=5000 train=5.30901 val=0.30776 pset=500.0 mset=272.4 mfrac=1.00
2023-07-20T10:56:59.922 EarlyStoppingCallback: 0.30776 best error at 5000 trials
2023-07-20T10:57:00.530 trials=10000 train=0.28302 val=0.24707 pset=500.0 mset=296.3 mfrac=1.00
2023-07-20T10:57:00.533 EarlyStoppingCallback: 0.24707 best error at 10000 trials
2023-07-20T10:57:01.377 trials=15000 train=0.23101 val=0.24616 pset=500.0 mset=468.8 mfrac=1.00
2023-07-20T10:57:01.380 EarlyStoppingCallback: 0.24616 best error at 15000 trials
2023-07-20T10:57:02.493 trials=20000 train=0.22959 val=0.24242 pset=500.0 mset=473.6 mfrac=1.00
2023-07-20T10:57:02.497 EarlyStoppingCallback: 0.24242 best error at 20000 trials
2023-07-20T10:57:03.559 trials=25000 t

[I 2023-07-20 10:57:23,448] Trial 1 finished with value: 0.16811090602716242 and parameters: {'e0': 0.09, 'beta': 0.03, 'n_init': 11}. Best is trial 1 with value: 0.16811090602716242.


2023-07-20T10:57:23.365 trials=100000 train=0.14687 val=0.17042 pset=500.0 mset=417.1 mfrac=0.49
2023-07-20T10:57:23.366 EarlyStoppingCallback: stopping
2023-07-20T10:57:23.367 EarlyStoppingCallback: restoring system from trial 75000 with error=0.16811
2023-07-20T10:57:24.546 trials=5000 train=7.28603 val=0.39324 pset=500.0 mset=244.8 mfrac=1.00
2023-07-20T10:57:24.549 EarlyStoppingCallback: 0.39324 best error at 5000 trials
2023-07-20T10:57:25.608 trials=10000 train=0.46325 val=0.29229 pset=500.0 mset=256.4 mfrac=1.00
2023-07-20T10:57:25.611 EarlyStoppingCallback: 0.29229 best error at 10000 trials
2023-07-20T10:57:27.125 trials=15000 train=0.34270 val=0.23230 pset=500.0 mset=420.3 mfrac=1.00
2023-07-20T10:57:27.127 EarlyStoppingCallback: 0.23230 best error at 15000 trials
2023-07-20T10:57:28.873 trials=20000 train=0.30204 val=0.23225 pset=500.0 mset=412.7 mfrac=0.39
2023-07-20T10:57:28.875 EarlyStoppingCallback: 0.23225 best error at 20000 trials
2023-07-20T10:57:30.536 trials=25000 

...

2023-07-20T11:03:14.672 trials=150000 train=0.08540 val=0.10835 pset=500.0 mset=483.0 mfrac=1.00
2023-07-20T11:03:14.673 EarlyStoppingCallback: stopping
2023-07-20T11:03:14.675 EarlyStoppingCallback: restoring system from trial 125000 with error=0.10599
2023-07-20T11:03:15.293 trials=5000 train=0.67527 val=0.61781 pset=500.0 mset=255.5 mfrac=0.56
2023-07-20T11:03:15.295 EarlyStoppingCallback: 0.61781 best error at 5000 trials
2023-07-20T11:03:15.755 trials=10000 train=0.59852 val=0.60439 pset=500.0 mset=276.1 mfrac=0.61
2023-07-20T11:03:15.756 EarlyStoppingCallback: 0.60439 best error at 10000 trials
2023-07-20T11:03:16.331 trials=15000 train=0.60919 val=0.60372 pset=500.0 mset=417.1 mfrac=0.33
2023-07-20T11:03:16.333 EarlyStoppingCallback: 0.60372 best error at 15000 trials
2023-07-20T11:03:17.009 trials=20000 train=0.61233 val=0.69736 pset=500.0 mset=412.5 mfrac=1.00
2023-07-20T11:03:17.676 trials=25000 train=0.59413 val=0.62148 pset=500.0 mset=397.5 mfrac=1.00
2023-07-20T11:03:18.34

[I 2023-07-20 11:03:19,677] Trial 8 finished with value: 0.6037161920886245 and parameters: {'e0': 0.12, 'beta': 0.17, 'n_init': 1}. Best is trial 3 with value: 0.09871648382509106.


2023-07-20T11:03:19.625 trials=40000 train=0.58512 val=0.64819 pset=500.0 mset=333.5 mfrac=0.00
2023-07-20T11:03:19.626 EarlyStoppingCallback: stopping
2023-07-20T11:03:19.626 EarlyStoppingCallback: restoring system from trial 15000 with error=0.60372
2023-07-20T11:03:21.159 trials=5000 train=11.82799 val=0.33387 pset=500.0 mset=275.6 mfrac=1.00
2023-07-20T11:03:21.163 EarlyStoppingCallback: 0.33387 best error at 5000 trials
2023-07-20T11:03:22.671 trials=10000 train=0.41403 val=0.27982 pset=500.0 mset=315.1 mfrac=1.00
2023-07-20T11:03:22.674 EarlyStoppingCallback: 0.27982 best error at 10000 trials
2023-07-20T11:03:24.872 trials=15000 train=0.31970 val=0.26878 pset=500.0 mset=470.0 mfrac=1.00
2023-07-20T11:03:24.874 EarlyStoppingCallback: 0.26878 best error at 15000 trials
2023-07-20T11:03:27.409 trials=20000 train=0.53180 val=0.55783 pset=500.0 mset=470.1 mfrac=1.00
2023-07-20T11:03:30.013 trials=25000 train=0.39943 val=0.22238 pset=500.0 mset=468.0 mfrac=1.00
2023-07-20T11:03:30.016

[I 2023-07-20 11:05:14,331] Trial 9 finished with value: 0.1121574869349999 and parameters: {'e0': 0.08, 'beta': 0.03, 'n_init': 91}. Best is trial 3 with value: 0.09871648382509106.


2023-07-20T11:05:14.156 trials=190000 train=0.09554 val=0.11314 pset=500.0 mset=495.0 mfrac=1.00
2023-07-20T11:05:14.157 EarlyStoppingCallback: stopping
2023-07-20T11:05:14.158 EarlyStoppingCallback: restoring system from trial 165000 with error=0.11216


In [12]:
best_trial = study.best_trial
print(f"Best trial - Evaluation metric: {best_trial.value:.4f}")
print(f"Best trial - Hyperparameters: {best_trial.params}")

Best trial - Evaluation metric: 0.0987
Best trial - Hyperparameters: {'e0': 0.1, 'beta': 0.09, 'n_init': 71}


In [ ]:
plot_param_importances(study)